# Hard spheres in 2D

In [1]:
import numpy as np
import plotly.offline as py
import plotly.tools as py_tools
from plotly.graph_objs import *
from scipy.optimize import minimize_scalar
from concurrent.futures import ProcessPoolExecutor

py.init_notebook_mode(connected=True)

%load_ext cython

In [2]:
%%cython

import numpy as np
cimport numpy as np
from libc.stdlib cimport rand, RAND_MAX

cpdef bint c_collision(np.ndarray x, np.ndarray config, double radius, int x_pos=-1):
    """Return True if the disk in position x collide with any other in config."""
    cdef np.ndarray distances = np.sum(np.power(config - x, 2), axis=1)
    
    if x_pos >= 0:
        distances[x_pos] = np.inf
        
    return np.any(distances < (2*radius)**2)


cpdef bint c_clone_collision(np.ndarray x, np.ndarray config, double radius, double system_size):
    """Check if the disk is in the area in which his periodic clones can undergo collisions."""

    cdef np.ndarray position = (x < 2*radius) -1*(x > system_size - 2*radius)
    cdef np.ndarray clone, clone1, clone2
    
    if np.sum(np.abs(position)) > 0:
        clone = x + system_size*position
        if c_collision(clone, config, radius):
            return True
    
        if np.sum(np.abs(position)) == 2:
            clone1 = x + system_size*np.array([0, position[1]])
            clone2 = x + system_size*np.array([position[0], 0])
            if c_collision(clone1, config, radius) or c_collision(clone2, config, radius):
                return True

    return False



In [3]:
def collision(new_position, state, disk, radius, sys_size):
    return (c_collision(new_position, state, radius, disk)
            or c_clone_collision(new_position, state, radius, sys_size))

def direct_sampling(disks, radius, sys_size):
    """Direct uniform extraction of non-overlapping disks in a square."""
    state = np.empty((disks, 2))
    rejected = 0
    i = 0
    while i < disks:
        x = np.random.uniform(0, sys_size, size=2)
        
        if collision(x, state[:i+1], i, radius, sys_size):
            rejected += 1
            i = 0
            continue
            
        state[i] = x
        i += 1

    acceptance =  disks / (rejected + disks)
        
    return state, acceptance

def density(disks, radius, sys_size):
    return (disks * radius**2 * np.pi)/(sys_size**2)

def generate_square_lattice(system_size, num_disks):
    """Generates a square lattice with a given number of disks."""
    ss_disks = np.sqrt(num_disks)
    step = system_size / ss_disks
    x = [ step*(.5 + i) for i in range(int(ss_disks))]
    
    return np.transpose([np.tile(x, len(x)), np.repeat(x, len(x))])


def disk_shape(xc, yc, radius):
    x0 = xc - radius
    y0 = yc - radius
    x1 = xc + radius
    y1 = yc + radius

    return {
        'type': 'circle',
        'xref': 'x',
        'yref': 'y',
        'fillcolor': '#1F77B4',
        'line': dict(width=0),
        'x0': x0,
        'y0': y0,
        'x1': x1,
        'y1': y1,
    }


def plot_state(state, radius, sys_size):
    """Plot the configuration of the system."""
    x, y = state.swapaxes(0, 1)
    trace = Scatter(
        x=x,
        y=y,
        mode='markers'
    )

    shapes = [{'type': 'square', 'x0': 0, 'y0': 0, 'x1': sys_size, 'y1': sys_size, 'xref':'x', 'yref':'y'}]
    
    for i in range(len(x)):
        shapes.append(disk_shape(x[i], y[i], radius))
    
    lyt = Layout(
        yaxis=dict(scaleanchor="x", showgrid=False),
        xaxis=dict(showgrid=False),
        shapes=shapes,
        hovermode="closest"
    )
    
    

    fig = dict(data=[trace], layout=lyt)

    py.iplot(fig)


## Direct sampling

We first try to implement a direct sampling strategy. We expect that reaching large density will be practically impossible, since the probability of obtaining two overlapping disk in the extracted configuration will be very high (the acceptance will tend to zero).

In [4]:
num_disks = 45
sys_size = 40
radius = 1.

state, acceptance = direct_sampling(num_disks, radius, sys_size)

plot_state(state, radius, sys_size)
print("For a density of disks of {:4f} we get an acceptance ratio of {:f}". \
      format(density(num_disks, radius, sys_size), acceptance))

For a density of disks of 0.088357 we get an acceptance ratio of 0.129310


As we can see even for small density the computational cost of obtaining a single configuration is very high. Also the presence of the periodic boundary condictions (PBCs) increases the computational cost. In fact, for each step, we have to check if the new disk extracted collides with any other (due to the PBCs, if a disk go beyond the square border, it can collide with the disks near the other side of the square). In this first approach we have controlled for each step if any of the particles duplicate (generated by the PBCs) collided with any other particle. In the following we'll do better, doing this check only when the disk is sufficiently near to the border (less than 2 times its radius).

## Markov chain Monte Carlo


In [9]:
def calculate_system_size(disks, radius, density):
    """Calculate the system size required to match a given density."""
    return np.sqrt(disks * radius**2 * np.pi / density)

def distances(x, config, x_pos):
    """Return the array of distances of x from any other disk in config."""
    distances = np.sqrt(np.sum(np.power(config - x, 2), axis=1))
    distances[x_pos] = np.inf
    
    return distances

def neighbours(x, config, x_pos, radius, system_size):
    """Compute the neighbours list (distance 2.8*disk radius from x)."""

    dist_min = distances(x, config, x_pos)

    position = (x < 2*radius) -1*(x > system_size - 2*radius)


    if np.sum(np.abs(position)):
        clone = x + system_size*position
        dist = distances(clone, config, x_pos)
        dist_min = np.minimum(dist_min, dist)
        k = np.where(dist == dist_min)


    if np.sum(np.abs(position)) == 2:
        clone1 = x + system_size*np.array([0, position[1]])
        clone2 = x + system_size*np.array([position[0], 0])
        dist1 = distances(clone1, config, x_pos)
        dist2 = distances(clone2, config, x_pos)
        dist_min = np.minimum(dist_min, np.minimum(dist1, dist2))


    dist_min[x_pos] = np.inf

    neighbours, = np.where(dist_min < 2.8*radius)

    pos = np.zeros((len(neighbours), 2))

    if np.sum(np.abs(position)):

        for i in range(len(neighbours)):

            if dist_min[neighbours[i]] == dist[neighbours[i]]:
                pos[i] = position
            elif np.sum(np.abs(position)) == 2:
                if dist_min[neighbours[i]] == dist1[neighbours[i]]:
                    pos[i] = np.array([0, position[1]])
                elif dist_min[neighbours[i]] == dist2[neighbours[i]]:
                    pos[i] = np.array([position[0], 0])

    return neighbours, pos


def psi_6(i, config, neighbour_list, pos, system_size):
    z = config[i] + pos*system_size - config[neighbour_list]
    
    if len(z) <= 0:
        return 0.
    
    with np.errstate(divide="ignore"):
        angles = np.arctan(z[:, 1] / z[:, 0])
        
    return np.sum(np.exp(6j*angles)) / len(neighbour_list)

In [10]:
class Simulation(object):
    def __init__(self, disks, radius, sys_size, step_size=None):
        """Defines a 2d disk simulation on a square space."""
        self.disks = disks
        self.radius = radius
        self.sys_size = sys_size
        self.step_size = step_size if step_size is not None else 0.5 * radius
        self.state = self._init_state()  # Initial configuration
        self.steps = 0
        self.accepted = 0
    
    def run(self, num_steps):
        """Run multiple steps (cumulatively)."""
        # Prepare all the random values we need at once (numpy performs better).
        disks = np.random.randint(self.disks, size=num_steps)
        steps = np.random.uniform(-self.step_size, +self.step_size, size=(num_steps, 2))
        
        for i in range(num_steps):
            self.move(disks[i], steps[i])
        
        return self.state
    
    def move(self, disk, step):
        """Try to move a random disk of the current state."""
        new_position = (self.state[disk] + step) % self.sys_size  # Periodic boundary conditions        
                
        if not self._collision(disk, new_position):
            self.state[disk] = new_position
            self.accepted += 1

        self.steps += 1
        
        return self.state
        
        
    def _collision(self, disk, new_position):
        """Detect a collision."""
        return (c_collision(new_position, self.state, self.radius, disk)
               or c_clone_collision(new_position, self.state, self.radius, self.sys_size))
            
            
    def _init_state(self):
        """Generate a square lattice initial state with the disks."""
        
        return generate_square_lattice(self.sys_size, self.disks)
    
    def reset(self):
        self.state = self._init_state()
        self.steps = 0
        self.accepted = 0

    def optimize_step_size(self):
        """Optimize the step size for the current system to reach an acceptance ratio of ~0.5."""
        test_instance = self.__class__(self.disks, self.radius, self.sys_size, self.radius*0.5)
        
        # Wrapper
        def test_acceptance(step_size, steps=5000):
            test_instance.reset()
            test_instance.step_size = step_size
            test_instance.run(steps)
            
            return abs(test_instance.accepted / test_instance.steps - 0.5)
        
        res = minimize_scalar(test_acceptance)
        if not res.success:
            raise Exception("Optimization of step size did not converge!")
            
        self.step_size = res.x
        
        return self.step_size
    
    def psi(self):
        psi = np.empty(self.disks, dtype=complex)
        for i in range(self.disks):
            neighbour_list, positions = neighbours(self.state[i], self.state, i, self.radius, self.sys_size)
            
            psi[i] = psi_6(i, self.state, neighbour_list, positions, self.sys_size)
        
        return np.sum(psi)


In [11]:
def plot_sequence(sequence):
    """Plot a sequence of states."""
    titles = list(map(lambda item: "{:.0e} steps (𝜓 = {:.2f})".format(item["steps"], item["psi"]), sequence["states"]))
    fig = py_tools.make_subplots(rows=1, cols=len(sequence["states"]), subplot_titles=titles, print_grid=False)
    lyt = Layout(
        hovermode="closest",
        showlegend=False,
        title="Density {:.2f}".format(sequence["density"])
    )
    shapes = []

    for i, item in enumerate(sequence["states"]):
        x, y = item["state"].swapaxes(0, 1)
        
        yaxis = "yaxis{}".format(i + 1)
        xaxis = "xaxis{}".format(i + 1)
        xref = "x{}".format(i + 1)
        yref = "y{}".format(i + 1)

        trace = Scatter(x=x, y=y, mode="markers", name="{:.0e} steps".format(item["steps"]),
                        marker=dict(color="#1F77B4"))
        fig.append_trace(trace, 1, i + 1)
            
        for i in range(len(x)):
            disk = disk_shape(x[i], y[i], sequence["simulation"].radius)
            disk["xref"] = xref
            disk["yref"] = yref
            shapes.append(disk)
    
        shapes.append({
            'type': 'square', 'x0': 0, 'y0': 0,
            'x1': sequence["simulation"].sys_size,
            'y1': sequence["simulation"].sys_size,
            'xref': xref, 'yref': yref
        })
        
        lyt.update({
            yaxis: {"scaleanchor": "x", "range": [0, sequence["simulation"].sys_size]},
            xaxis: {"range": [0, sequence["simulation"].sys_size]}
        })
                    
    lyt.update({"shapes": shapes})
    fig.layout.update(lyt)
    py.iplot(fig)

In [15]:
DISK_RADIUS = 1.
NUM_DISKS = 16**2
NUM_STEPS = 10**8

densities = [0.5, 0.6, 0.72]

def run_simulation(density):
    sys_size = calculate_system_size(NUM_DISKS, DISK_RADIUS, density)
    sim = Simulation(NUM_DISKS, DISK_RADIUS, sys_size)
    sim.optimize_step_size()  # magic!
    states = [{"state": sim.state.copy(), "steps": 0, "psi": sim.psi()}]
    for _ in range(2):
        sim.run(NUM_STEPS // 2)
        
        states.append({
            "state": sim.state.copy(),
            "steps": sim.steps,
            "psi": sim.psi()
        })
    
    return {
        "density": density,
        "states": states,
        "simulation": sim
    }


with ProcessPoolExecutor() as executor:
    sequences = executor.map(run_simulation, densities)
    
for s in list(sequences):
    plot_sequence(s)

For the a density of 0.5 nothing particular happens: the system gets disordered and then remains disordered, behaving like a fluid.
When the density increases we can see that, after a sufficient time, domains of ordered (aligned) particles arise in the system. In particular, the final configuration tends to the optimum packing (hexagonal close packing).